In [1]:
from datetime import datetime
import os
import cv2
path = 'cars-and-trucks/'

In [2]:
def get_filename():
    now = datetime.now()
    time_stamp = now.strftime("%m-%d-%Y %H:%M:%S")
    file_name = time_stamp + '.mp4'
    return (os.path.join(path, file_name))

In [3]:
def look_for_cars():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
            print('Error - could not get the camera opened')
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320) # supported 160x90 or 320x180
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 180)
    # for some reason this takes forever to start up now.        
    car_cascade = cv2.CascadeClassifier('cars.xml')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    recording = False
    frames_with_cars = 0
    frames_since_recording = 0
    finished = False
    while not finished:
        # reads frame from a video
        ret, frame = cap.read() # does every frame get sent to the file?
        if ret: # if you received a frame, then proceed
            if recording:
                frames_since_recording += 1
            if not recording:
                cv2.imshow('Car Detection', frame)
            # convert to gray scale of each frame before doing the cascade
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Detects cars of different sizes in the input image
            cars = car_cascade.detectMultiScale(gray, 1.1, 2)
            # Draw a rectangle of each car in the frame
            for (x,y,w,h) in cars:
                cv2.rectangle(gray,(x,y),(x+w,y+h),(0,0,255),2)
                # Display frame in a window
                cv2.imshow('Car Detection', gray)
                frames_with_cars += 1
                if not recording:
                    video_file = get_filename()
                    print('Starting to record to file: ',video_file)
                    writer = cv2.VideoWriter(video_file, 
                                     cv2.VideoWriter_fourcc(*'mp4v'),
                                     30, (width, height))
                    recording = True
            if recording:
                writer.write(frame)
            # If you have at least 60 frames since you first detected a car
            finished = (frames_since_recording - frames_with_cars > 30) 
        if cv2.waitKey(1) == 27:
            finished = True
            break
    cap.release()
    if recording:
        writer.release()
    cv2.destroyAllWindows()
    print('frames_with_cars: ',frames_with_cars)
    print('frames_since_recording started: ', frames_since_recording)
    return(finished)

In [ ]:
video_count = 0
while (video_count < 50):
    result = look_for_cars()
    video_count += 1

Starting to record to file:  cars-and-trucks/08-17-2021 11:00:36.mp4
frames_with_cars:  44
frames_since_recording started:  75
Starting to record to file:  cars-and-trucks/08-17-2021 11:00:46.mp4
frames_with_cars:  59
frames_since_recording started:  90
Starting to record to file:  cars-and-trucks/08-17-2021 11:00:54.mp4
frames_with_cars:  55
frames_since_recording started:  86
Starting to record to file:  cars-and-trucks/08-17-2021 11:01:25.mp4
frames_with_cars:  61
frames_since_recording started:  92
Starting to record to file:  cars-and-trucks/08-17-2021 11:01:29.mp4
frames_with_cars:  19
frames_since_recording started:  50
Starting to record to file:  cars-and-trucks/08-17-2021 11:04:24.mp4
frames_with_cars:  71
frames_since_recording started:  102
Starting to record to file:  cars-and-trucks/08-17-2021 11:06:35.mp4
frames_with_cars:  38
frames_since_recording started:  69
Starting to record to file:  cars-and-trucks/08-17-2021 11:07:34.mp4
frames_with_cars:  45
frames_since_record

In [ ]:
# escape no longer works.
# I think the next problem is due to the capture of the frame stops, or 
# it writes the same frame to the file twice, or more.
#
# It seems to be that it records the first 1/4 second, then slows quite a bit.
# So it's about 8-10 frames in, then slows down, but picks up after about 30 frames. 
#
# If I can average the speed over the frame, I think I can overcome the issue.
# but it is annoying. What am I saving with each frame to the log?
# just frames_with_cars and frames_since_recording (started)